In [6]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

In [19]:
# Set seed
SEED = 10
np.random.seed(SEED)

# Load dữ liệu
df = pd.read_csv('C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv')
features = [
    'Transaction Hash_len', 'Original_len', 'signature_len',
    'From_len', 'To_len', 'sender_len', 'paymaster_len',
    'Txn Fee', 'logIndex', 'actualGasCost',
    'actualGasUsed', 'nonce', 'success', 'Blockno', 'DateTime_ts'
]
X = df[features].astype(float)
y = df['Gas Used'].astype(float)

# Chia tập train/validation (không bắt buộc cho K-Fold nhưng vẫn làm cho đồng nhất)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

# Scale dữ liệu
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = pd.DataFrame(scaler_X.fit_transform(X), columns=X.columns)
y_scaled = pd.Series(scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten(), name='Gas Used')

# K-Fold
kf = KFold(n_splits=10, shuffle=True, random_state=SEED)

# Đánh giá
mse_scores, mae_scores, rmse_scores, r2_scores = [], [], [], []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_scaled), start=1):
    X_tr = X_train_scaled.iloc[train_idx]
    y_tr = y_train_scaled[train_idx]
    X_val = X_train_scaled.iloc[val_idx]
    y_val = y_train_scaled[val_idx]

    model = LGBMRegressor()
             # <- mở nếu LightGBM của bạn hỗ trợ GP )

    # Early stopping dùng callback (tương thích nhiều version LightGBM)
    callbacks = [
        lgb.early_stopping(stopping_rounds=200, verbose=False),
        lgb.log_evaluation(period=0)  # tắt log mỗi vòng
    ]

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=callbacks
    )

    # Dự đoán dùng best_iteration_ (nếu có)
    y_pred = model.predict(X_val, num_iteration=getattr(model, "best_iteration_", None))

    # Metrics (trên thang đã scale)
    mse  = mean_squared_error(y_val, y_pred)
    mae  = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_val, y_pred)

    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

  
    print(f"Fold {fold} - MSE: {mse:.6f} | MAE: {mae:.6f} | RMSE: {rmse:.6f} | R²: {r2:.6f}")

# Tổng kết sau tất cả folds
print("\n=== Kết quả trung bình (scaled) ===")
print(f"Avg MSE : {np.mean(mse_scores):.6f}")
print(f"Avg MAE : {np.mean(mae_scores):.6f}")
print(f"Avg RMSE: {np.mean(rmse_scores):.6f}")
print(f"Avg R²  : {np.mean(r2_scores):.6f}")

Fold 1 - MSE: 0.000012 | MAE: 0.001112 | RMSE: 0.003475 | R²: 0.978630
Fold 2 - MSE: 0.000037 | MAE: 0.001163 | RMSE: 0.006112 | R²: 0.929798
Fold 3 - MSE: 0.000011 | MAE: 0.001090 | RMSE: 0.003306 | R²: 0.978842
Fold 4 - MSE: 0.000121 | MAE: 0.002027 | RMSE: 0.010989 | R²: 0.765615
Fold 5 - MSE: 0.000020 | MAE: 0.001200 | RMSE: 0.004442 | R²: 0.962890
Fold 6 - MSE: 0.000009 | MAE: 0.001080 | RMSE: 0.003069 | R²: 0.979434
Fold 7 - MSE: 0.000023 | MAE: 0.001134 | RMSE: 0.004780 | R²: 0.958268
Fold 8 - MSE: 0.000011 | MAE: 0.001115 | RMSE: 0.003348 | R²: 0.977417
Fold 9 - MSE: 0.000045 | MAE: 0.001273 | RMSE: 0.006745 | R²: 0.921046
Fold 10 - MSE: 0.000015 | MAE: 0.001130 | RMSE: 0.003920 | R²: 0.968802

=== Kết quả trung bình (scaled) ===
Avg MSE : 0.000031
Avg MAE : 0.001232
Avg RMSE: 0.005019
Avg R²  : 0.942074


In [ ]:
# -------------------- Cấu hình --------------------
SEED   = 10        # theo code bạn dùng lúc trước
epochs = 20        # đổi 30 nếu muốn
k      = 10

# -------------------- Load & chọn dữ liệu --------------------
df = pd.read_csv(r"C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv")

features = [
    'Transaction Hash_len', 'Original_len', 'signature_len',
    'From_len', 'To_len', 'sender_len', 'paymaster_len',
    'Txn Fee', 'logIndex', 'actualGasCost',
    'actualGasUsed', 'nonce', 'success', 'Blockno', 'DateTime_ts'
]
target = 'Gas Used'

# ép kiểu số và loại NaN
X = df[features].apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(df[target], errors='coerce')
mask = X.notna().all(axis=1) & y.notna()
X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

# (Tuỳ chọn) nếu muốn bỏ cảnh báo "whitespace in feature_names", đổi tên cột:
# X = X.rename(columns=lambda c: c.replace(" ", "_"))

# Giữ lại 1 hold-out nếu bạn cần về sau (không dùng trong CV bên dưới)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

# đảm bảo pandas để .iloc
X_train_df = X_train if isinstance(X_train, pd.DataFrame) else pd.DataFrame(X_train)
y_train_sr = y_train if isinstance(y_train, pd.Series) else pd.Series(y_train, dtype=float)

# -------------------- Vòng lặp epochs × kfold --------------------
cats = np.zeros((epochs, 4))  # mỗi hàng: [RMSE, MSE, MAE, R2] trên thang scaled

for ep in range(epochs):
    kf = KFold(n_splits=k, shuffle=True, random_state=SEED + ep)

    rmse_list, mse_list, mae_list, r2_list = [], [], [], []

    for train_idx, val_idx in kf.split(X_train_df):
        # tách fold (raw)
        X_tr_raw, X_va_raw = X_train_df.iloc[train_idx], X_train_df.iloc[val_idx]
        y_tr_raw, y_va_raw = y_train_sr.iloc[train_idx], y_train_sr.iloc[val_idx]

        # MinMax scale TRONG MỖI FOLD (tránh leakage)
        sx = MinMaxScaler()
        sy = MinMaxScaler()

        X_tr = sx.fit_transform(X_tr_raw)
        X_va = sx.transform(X_va_raw)

        y_tr = sy.fit_transform(y_tr_raw.to_numpy().reshape(-1, 1)).ravel()
        y_va = sy.transform(y_va_raw.to_numpy().reshape(-1, 1)).ravel()

        # Model LightGBM + early stopping qua callbacks
        model = LGBMRegressor(
            random_state=SEED,
            n_estimators=10000,      # đủ lớn để early stopping cắt sớm
            learning_rate=0.05,
            n_jobs=-1,
            force_col_wise=True,
            verbose=-1
        )
        callbacks = [
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0)  # tắt log theo vòng
        ]

        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric='l2',        # MSE trên thang scaled
            callbacks=callbacks
        )

        # dự đoán với best_iteration_ (nếu có)
        best_iter = getattr(model, "best_iteration_", None)
        y_pred = model.predict(X_va, num_iteration=best_iter)

        # ---- Metrics trên thang MinMax-scaled của y ----
        mse  = mean_squared_error(y_va, y_pred)
        rmse = np.sqrt(mse)
        mae  = mean_absolute_error(y_va, y_pred)
        r2   = r2_score(y_va, y_pred)

        rmse_list.append(rmse)
        mse_list.append(mse)
        mae_list.append(mae)
        r2_list.append(r2)

    # lưu mean metrics của epoch này (qua 10 folds)
    cats[ep, :] = [
        np.mean(rmse_list),
        np.mean(mse_list),
        np.mean(mae_list),
        np.mean(r2_list)
    ]

# -------------------- Trung bình qua 20 epochs --------------------
final_mean = np.mean(cats, axis=0)

print("Final mean on MinMax-scaled y [RMSE, MSE, MAE, R2]:")
print(final_mean)  # 1 dòng mean cuối cùng
print("\nPer-epoch metrics (rows) on MinMax-scaled y [RMSE, MSE, MAE, R2]:")
print(cats)        # bảng (epochs × 4)